# 2. Modelling SVR Linear

---

In [15]:
## load modules and run mlflow_logging.ipynb to get function to track model information on MLFLow
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import LinearSVR
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.model_selection import GridSearchCV

from modeling.functions import modelling, log_to_mlflow, get_features, save_models, save_results

In [2]:
data = pd.read_csv('../data/GEFCom2014Data/Wind/raw_data_incl_features.csv', parse_dates=['TIMESTAMP'])
data.head()
data.dropna(inplace=True)
data.info()
RSEED = 42

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175265 entries, 0 to 175433
Data columns (total 19 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   ZONEID      175265 non-null  int64         
 1   TIMESTAMP   175265 non-null  datetime64[ns]
 2   TARGETVAR   175265 non-null  float64       
 3   U10         175265 non-null  float64       
 4   V10         175265 non-null  float64       
 5   U100        175265 non-null  float64       
 6   V100        175265 non-null  float64       
 7   HOUR        175265 non-null  int64         
 8   MONTH       175265 non-null  int64         
 9   WEEKDAY     175265 non-null  int64         
 10  IS_HOLIDAY  175265 non-null  int64         
 11  WS10        175265 non-null  float64       
 12  WS100       175265 non-null  float64       
 13  WD10        175265 non-null  float64       
 14  WD100       175265 non-null  float64       
 15  WD100CARD   175265 non-null  object        
 16  WD

In [3]:
data = pd.get_dummies(data, columns = ['WD100CARD','WD10CARD'])
data.head()

,ZONEID,TIMESTAMP,TARGETVAR,U10,V10,U100,V100,HOUR,MONTH,WEEKDAY,...,WD10CARD_NNW,WD10CARD_NW,WD10CARD_S,WD10CARD_SE,WD10CARD_SSE,WD10CARD_SSW,WD10CARD_SW,WD10CARD_W,WD10CARD_WNW,WD10CARD_WSW
0,1,2012-01-01 01:00:00,0.000000,2.124600,-2.681966,2.864280,-3.666076,1,1,6,...,0,1,0,0,0,0,0,0,0,0
1,1,2012-01-01 02:00:00,0.054879,2.521695,-1.796960,3.344859,-2.464761,2,1,6,...,0,1,0,0,0,0,0,0,0,0
2,1,2012-01-01 03:00:00,0.110234,2.672210,-0.822516,3.508448,-1.214093,3,1,6,...,0,0,0,0,0,0,0,0,1,0
3,1,2012-01-01 04:00:00,0.165116,2.457504,-0.143642,3.215233,-0.355546,4,1,6,...,0,0,0,0,0,0,0,1,0,0
4,1,2012-01-01 05:00:00,0.156940,2.245898,0.389576,2.957678,0.332701,5,1,6,...,0,0,0,0,0,0,0,1,0,0


In [4]:
## train-test-split
#data_train, data_test = train_test_split(data, test_size=0.25, random_state=RSEED, stratify=data.ZONEID)
data_train = data[data.TIMESTAMP <= '2013-07-01 00:00:00']
data_test = data[data.TIMESTAMP > '2013-07-01 00:00:00']

In [5]:
# define features and feature dict
feature_dict = get_features(data)

features = feature_dict['all']

In [6]:
# define zone
zone = 1

In [7]:
# split train and test data in feature and TARGETVAR parts and cut data to desired zones
X_train = data_train[data_train.ZONEID == zone][features]
y_train = data_train[data_train.ZONEID == zone].TARGETVAR

X_test = data_test[data_test.ZONEID == zone][features]
y_test = data_test[data_test.ZONEID == zone].TARGETVAR

In [8]:
# Scale data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
model = LinearSVR()

param_grid = {'C': [0.01, 0.1, 1, 10, 100]}

# cv = GridSearchCV(model, param_grid=param_grid)
# cv.fit(X_train, y_train)

In [10]:
#y_pred = cv.predict(X_test)

#print(mean_squared_error(y_test, y_pred, squared=False))

In [13]:
model = LinearSVR(max_iter=100000)
scaler = MinMaxScaler()

model_dict = {}
results_train = {}
results_test = {}
param_grid = {'C': [0.1, 1, 10]}

for key in feature_dict.keys():
    print(f'Features: {key}')
    results_train[key],results_test[key], model_dict[key] = modelling(data_train, data_test, feature_dict[key], model = model, scaler=scaler, print_scores=True, log=None, infotext_mlflow=None, save_model= True, perform_gridCV= True, param_grid= param_grid, n_jobs = 3)
    results_test[key] = {k : np.round(value,5) for k,value in results_test[key].items()}

Features: all
Scaler: MinMaxScaler
Scaled X_train min/max: 0.0, 1.0
Scaled X_test min/max: -0.0, 1.0

ZONEID 1
Fitting 5 folds for each of 3 candidates, totalling 15 fits


/Users/msp/neuefische/Capstone_WindPowerPredicting/.venv/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/msp/neuefische/Capstone_WindPowerPredicting/.venv/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/msp/neuefische/Capstone_WindPowerPredicting/.venv/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


[CV] END ..............................................C=0.1; total time=   0.0s
[CV] END ..............................................C=0.1; total time=   0.0s
[CV] END ..............................................C=0.1; total time=   0.0s
[CV] END ..............................................C=0.1; total time=   0.0s
[CV] END ..............................................C=0.1; total time=   0.0s
[CV] END ................................................C=1; total time=   0.3s
[CV] END ................................................C=1; total time=   0.3s
[CV] END ................................................C=1; total time=   0.3s
[CV] END ................................................C=1; total time=   0.2s
[CV] END ................................................C=1; total time=   0.3s
[CV] END ...............................................C=10; total time=   1.5s
[CV] END ...............................................C=10; total time=   1.4s
[CV] END ...................

In [19]:
results_test
#save_models(model_dict)
save_results(results_train, results_test, '../saved_models/211203_1152_LinearSVR')

In [29]:
def save_scores(results_dict):
    pass




features = []
zones = []
train_score = []
test_score = []

for key in results_train.keys():
    for zone in results_train[key].keys():
        features.append(key)
        zones.append(zone)
        train_score.append(results_train[key][zone])
        test_score.append(results_test[key][zone])

df = pd.DataFrame({'features':features,'zone': zones,'train_score': train_score,'test_score': test_score})

df.to_csv(path + '/' + file_name, index=False)



In [23]:
df

,features,zone,train_score,test_score
0,all,ZONE1,0.183306,0.18496
1,all,ZONE2,0.153837,0.14979
2,all,ZONE3,0.151562,0.15334
3,all,ZONE4,0.176471,0.17835
4,all,ZONE5,0.177099,0.18119
...,...,...,...,...
138,no_deg_comp_ten,ZONE7,0.140510,0.14144
139,no_deg_comp_ten,ZONE8,0.171094,0.17041
140,no_deg_comp_ten,ZONE9,0.165482,0.16451
141,no_deg_comp_ten,ZONE10,0.202646,0.20443


In [24]:
df.zone.unique()

array(['ZONE1', 'ZONE2', 'ZONE3', 'ZONE4', 'ZONE5', 'ZONE6', 'ZONE7',
       'ZONE8', 'ZONE9', 'ZONE10', 'TOTAL'], dtype=object)

In [28]:
path = '../saved_models/211201_0915_GridSearchCV'

file_name = path.split('/')[-1] + '.csv'

file_name

'211201_0915_GridSearchCV.csv'